# SOURCE CODE LAMA

In [1]:
import tweepy
import pandas as pd
import re

In [ ]:
auth = tweepy.OAuthHandler("mAViUQaittIvlfAg99yY3yaSA", "SabL3v6MIsxkSTUTC4c1bhqodbss4pkrIJYz9dXld6txZVH0BL")
auth.set_access_token("1273402051817467904-VwNfdVmhlTWIyRFzIKzUAq1UX7CzoA", "ipMnMqhoSFNKzTU1dTeJOIkfLKalkfJumClH9CqiB0mFL")

api = tweepy.API(auth)

In [ ]:
keyword = ['jokowi', '3 periode', 'presiden', 'jabatan']

In [ ]:
dataTweets = api.search_tweets(q=', '.join(keyword) + ' -filter:retweets', max_id = str(last_id - 1), tweet_mode='extended', lang='id', count=1000)

tweets_time = []
tweets_username = []
tweets_content = []

for tweet in dataTweets:
    if tweet.user.followers_count > 30:
        print(str(tweet.created_at.date()) + ' '+ str(tweet.created_at.time()), tweet.full_text, tweet.user.screen_name)
        tweets_time.append(str(tweet.created_at.date()) + ' '+ str(tweet.created_at.time()))
        tweets_username.append(tweet.user.screen_name)
        tweets_content.append(tweet.full_text)

df = pd.DataFrame({'created_at': tweets_time, 
                   'username': tweets_username, 
                   'content': tweets_content})

df.to_excel('./Datap3p_2Raw_7.xlsx', sheet_name='Raw Data', index=False)

In [ ]:
openDatap3p = pd.read_excel('Datap3p_2Raw.xlsx')
openDatap3p.head()

In [ ]:
# Case Folding
def CaseFolding(comment):
    comment = comment.lower()
    return comment

openDatap3p['Tweets'] = openDatap3p['Tweets'].apply(CaseFolding)
openDatap3p.to_excel('./Datap3p_3Raw11.xlsx', sheet_name='Case Folding', index=False)
openDatap3p.head()

In [ ]:
# Cleansing
def Cleansing(comment):
    comment = comment.lower()
    comment = ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)"," ", comment).split())
    return comment

openDatap3p['content'] = openDatap3p['content'].apply(Cleansing)
openDatap3p.to_excel('./Datap3p_Cleansing.xlsx', index=False)
openDatap3p.head()

In [ ]:
# Tokenizing
def Token(comments):
    nstr = comments.split(' ')
    dat = []
    a = -1
    for hu in nstr:
        a = a + 1
    if hu == '':
        dat.append(a)
    p = 0
    b = 0
    for q in dat:
        b = q - p
        del nstr[b]
        p = p + 1
    return nstr

openDatap3p['content'] = openDatap3p['content'].apply(Token)
openDatap3p.to_excel('./Datap3p_Token.xlsx', index=False)
openDatap3p.head()

In [ ]:
openDatap3p

In [ ]:
data = [1,2,4,51,53,53,53,53,34,3,4]
[num for num in range(1, len(data))]

# SOURCE CODE BARU

In [1]:
import tweepy
import pandas as pd
from datetime import datetime, timedelta
import string

class CrawlingDataTwitter:
    def __init__(self):
        # Inisialisasi autentikasi tweepy
        self.auth = tweepy.OAuth1UserHandler(
            consumer_key = 'mAViUQaittIvlfAg99yY3yaSA',
            consumer_secret = 'SabL3v6MIsxkSTUTC4c1bhqodbss4pkrIJYz9dXld6txZVH0BL',
            access_token = '1273402051817467904-VwNfdVmhlTWIyRFzIKzUAq1UX7CzoA',
            access_token_secret = 'ipMnMqhoSFNKzTU1dTeJOIkfLKalkfJumClH9CqiB0mFL'
        )

        # Inisialisasi API untuk akses twitter RESTful API methods
        self.api = tweepy.API(self.auth)
        
    def search_data(self, keyword, num_tweets):
        # List untuk menampung waktu dibuatnya tweet, username tweet, text tweets dan Id tiap tweet
        time_list, username_list, tweet_list, tweet_id = [], [], [], []

        # Variabel penampung id terkecil
        last_id = 0 
        
        while len(tweet_list) < num_tweets:
            try:
                # Pengambilan tweet dengan operator -filter:retweets untuk menyaring data retweets
                # paramter count, untuk mengambil data sebanyak 1000 tweets
                # paramater max_id untuk mencegah data redundan dengan mengambil id terkecil setelah fetching data twitter
                # paramter lang='id' untuk mengambil bahasa twitter
                # paramter tweet_mode='extended' untuk mengambil text tweet secara penuh
                new_tweets = self.api.search_tweets(q = ', '.join(keyword) + ' -filter:retweets', count = num_tweets, max_id = last_id - 1, lang = 'id', tweet_mode = 'extended')
            except tweepy.errors.TweepyException as e:
                # Menghentikan proses pengambilan tweet jika terjadi error
                print(f"Error : {e}")
                break
            else:
                # Menghentikan proses pengambilan tweet jika tidak ada lagi tweet yang relevan dengan keyword
                if not new_tweets:
                    print(f"Could not find any more tweets!, total tweets : {len(tweet_list)}")
                    break
                else:
                    for tweet in new_tweets:
                        # Menyimpan format waktu ke dalam tuple
                        struct_time = datetime.strptime(
                            f'{tweet.created_at.date()}' + ' '+ f'{tweet.created_at.time()}',
                            '%Y-%m-%d %H:%M:%S')

                        # Konversi waktu UTC +00:00 ke UTC +07:00 (Indonesia bagian barat)
                        offset_hours = 7
                        local_time = struct_time + timedelta(hours=offset_hours)

                        # Mengubah struktur waktu menjadi format 24 jam (0 - 23)
                        final_timestamp =  datetime.strftime(local_time, '%Y-%m-%d %H:%M:%S')

                        # Menyimpan semua data pada tiap list
                        time_list.append(final_timestamp)
                        username_list.append(tweet.user.screen_name)
                        tweet_list.append(tweet.full_text)

                        # Menyimpan tiap Id tweets
                        tweet_id.append(tweet.id)

            # Update Id dengan mengambil Id terkecil
            last_id = min(tweet_id)
        
        df = pd.DataFrame({'Created_at':time_list,
                           'Username':username_list,
                           'Tweets':tweet_list})
        
        return df

In [2]:
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
import re

class PreprocessingText:
    def __init__(self):
        self.slang_list = [kamus.strip('\n').strip('\r') for kamus in open('../kamus/daftar_slangwords.txt')]
        self.stopword_list = [line.strip('\n')for line in open('../kamus/daftar_stopwords.txt')]
        
    # Case Folding
    def CaseFolding(self, tweets):
        tweets = tweets.lower()
        
        return tweets
    
    # Cleansing
    def Cleansing(self, tweets):
        # Menghilangkan mention, hashtag, character reference
        tweets = re.sub('[@#&][A-Za-z0-9_]+'," ", tweets)

        # Menghilangkan Tautan
        tweets = re.sub("\w+:\/\/\S+"," ", tweets)

        # Menghilangkan tanda baca
        tweets = re.sub('[()!?;,]', ' ', tweets)
        tweets = re.sub('\[.*?\]',' ', tweets)

        # Menghilangkan tanda selain huruf
        tweets = re.sub("[^a-z]"," ", tweets)

        # Menghilangkan spasi lebih dari 1
        tweets = ' '.join(tweets.split())
        
        return tweets
    
    # Replace Slangwords
    def Slangwords(self, tweets):
        for line in self.slang_list:
            slangword = line.strip().split(":")
            if slangword[0] in tweets.split(" "):
                tweets = tweets.replace(slangword[0] + " ", slangword[1] + " ")
                tweets = tweets.replace(" " + slangword[0], " " + slangword[1])
                
        return tweets
    
    # Stopwords Removal
    def StopwordsRemoval(self, tweets):
        stopwords_list = list(map(lambda x:x.strip(),list(self.stopword_list)))
        tweets = tweets.split()
        tweets = [w for w in tweets if not w in stopwords_list]
        tweets = " ".join(word for word in tweets)
        
        return tweets
    
    # Tokenization
    def Tokenization(self, tweets):
        tweets = tweets.split()
        
        return tweets
    
    # Normalization (Stemming)
    def Stemming(self, tweets):
        factory = StemmerFactory()
        stemmer = factory.create_stemmer()
        do = []
        for w in tweets:
            dt = stemmer.stem(w)
            do.append(dt)
        d_clean = []
        d_clean = " ".join(do)
        print(d_clean)
        
        return d_clean
    
    def Stemming1(self, tweets):
        stemmer = StemmerFactory.create_stemmer()
        tweets = stemmer.stem(tweets)
        print(tweets)

        return tweets

In [3]:
class LabellingData:
    def __init__(self):
        self.pos_kata= open("../kamus/kata_positif.txt","r").readlines()
        self.neg_kata= open("../kamus/kata_negatif.txt","r").readlines()  

    def Lexicon_based(self, tweet_list):
        sentiments = [] #Membuat sebuah list menyimpan nilai sentiment

        #fungsi menghitung sentiment setiap tweet
        for item in tweet_list:
            tweets = item.strip().split() #tokenization
            # print(tweets)

            count_p = 0 #nilai positif
            count_n = 0 #nilai negatif

            for tweet in tweets:
                for kata_pos in self.pos_kata:
                    if kata_pos.strip().lower() == tweet.lower():
                        # print(kata_pos, ['positif'])
                        count_p += 1
                for kata_neg in self.neg_kata:
                    if kata_neg.strip().lower() == tweet.lower():
                        # print(kata_neg, ['negatif'])
                        count_n += 1

            # print ("positif: "+str(count_p))
            # print ("negatif: "+str(count_n))
            sentiments.append(count_p - count_n)

        return sentiments

    def Analyze_score(self, score):
        if score < 0 or score == 0:
            return 'Negatif'
        # elif score == 0:
        #    return "Netral"
        else:
            return "Positif"

In [4]:
class MultiNB:
    def __init__(self,alpha=1):
        self.alpha = alpha
    
    def _prior(self): # CHECKED
        """
        Calculates prior for each unique class in y. P(y)
        """
        P = np.zeros((self.n_classes_))
        _, self.dist = np.unique(self.y,return_counts=True)
        for i in range(self.classes_.shape[0]):
            P[i] = self.dist[i] / self.n_samples
        return P
            
    def fit(self, X, y): # CHECKED, matches with sklearn
        """
        Calculates the following things- 
            class_priors_ is list of priors for each y.
            N_yi: 2D array. Contains for each class in y, the number of time each feature i appears under y.
            N_y: 1D array. Contains for each class in y, the number of all features appear under y.
            
        params
        ------
        X: 2D array. shape(n_samples, n_features)
            Multinomial data
        y: 1D array. shape(n_samples,). Labels must be encoded to integers.
        """
        self.y = y
        self.n_samples, self.n_features = X.shape
        self.classes_ = np.unique(y)
        self.n_classes_ = self.classes_.shape[0]
        self.class_priors_ = self._prior()
        
        # distinct values in each features
        self.uniques = []
        for i in range(self.n_features):
            tmp = np.unique(X[:,i])
            self.uniques.append( tmp )
            
        self.N_yi = np.zeros((self.n_classes_, self.n_features)) # feature count
        self.N_y = np.zeros((self.n_classes_)) # total count 
        for i in self.classes_: # x axis
            indices = np.argwhere(self.y==i).flatten()
            columnwise_sum = []
            for j in range(self.n_features): # y axis
                columnwise_sum.append(np.sum(X[indices,j]))
                
            self.N_yi[i] = columnwise_sum # 2d
            self.N_y[i] = np.sum(columnwise_sum) # 1d
            
    def _theta(self, x_i, i, h):
        """
        Calculates theta_yi. aka P(xi | y) using eqn(1) in the notebook.
        
        params
        ------
        x_i: int. 
            feature x_i
            
        i: int.
            feature index. 
            
        h: int or string.
            a class in y
        
        returns
        -------
        theta_yi: P(xi | y)
        """
        
        Nyi = self.N_yi[h,i]
        Ny  = self.N_y[h]
        
        numerator = Nyi + self.alpha
        denominator = Ny + (self.alpha * self.n_features)
        
        return  (numerator / denominator)**x_i
    
    def _likelyhood(self, x, h):
        """
        Calculates P(E|H) = P(E1|H) * P(E2|H) .. * P(En|H).
        
        params
        ------
        x: array. shape(n_features,)
            a row of data.
        h: int. 
            a class in y
        """
        tmp = []
        for i in range(x.shape[0]):
            tmp.append(self._theta(x[i], i,h))
        
        return np.prod(tmp)
    
    def predict(self, X):
        samples, features = X.shape
        self.predict_proba = np.zeros((samples,self.n_classes_))
        
        for i in range(X.shape[0]):
            joint_likelyhood = np.zeros((self.n_classes_))
            
            for h in range(self.n_classes_):
                joint_likelyhood[h]  = self.class_priors_[h] * self._likelyhood(X[i],h) # P(y) P(X|y) 
                
            denominator = np.sum(joint_likelyhood)
            
            for h in range(self.n_classes_):
                numerator = joint_likelyhood[h]
                self.predict_proba[i,h] = (numerator / denominator)
            
        indices = np.argmax(self.predict_proba,axis=1)
        return self.classes_[indices]

In [ ]:
%%time
key = ['sambo', 'polisi', 'josua']

crawldata = CrawlingDataTwitter()

df = crawldata.search_data(key, 50000)

df.to_excel('./DataPolri.xlsx', sheet_name='Raw Data', index=False)
print('Total data :', len(df))

: 

In [7]:
%%time
pretext = PreprocessingText()

# df_casefolding = df['Tweets'].apply(pretext.CaseFolding)
# df_cleansing = df_casefolding.apply(pretext.Cleansing).drop_duplicates()
# df_slangwords = df_cleansing.apply(pretext.Slangwords)
# df_stopwords = df_slangwords.apply(pretext.StopwordsRemoval)
# df_token = df_stopwords.apply(pretext.Tokenization)
# df_stemming = df_token.apply(pretext.Stemming)

# ======================================================================================================================
df = pd.read_excel('./Datap3p.xlsx', engine='openpyxl')

df_casefolding = df.copy()
df_casefolding['Tweets'] = df_casefolding['Tweets'].apply(pretext.CaseFolding)

df_cleansing = df_casefolding.copy()
df_cleansing['Tweets'] = df_cleansing['Tweets'].apply(pretext.Cleansing)
# df_cleansing = df_cleansing.drop_duplicates(subset='Tweets')

df_slangwords = df_cleansing.copy()
df_slangwords['Tweets'] = df_slangwords['Tweets'].apply(pretext.Slangwords)

df_stopwords = df_slangwords.copy()
df_stopwords['Tweets'] = df_stopwords['Tweets'].apply(pretext.StopwordsRemoval)

df_token = df_stopwords.copy()
df_token['Tweets'] = df_token['Tweets'].apply(pretext.Tokenization)

df_stemming = df_token.copy()
df_stemming['Tweets'] = df_stemming['Tweets'].apply(pretext.Stemming)

dataAll = {'Raw Data': df, 'Case Folding': df_casefolding, 'Cleansing': df_cleansing, 
           'Normalization': df_slangwords, 'StopWords Removal': df_stopwords, 'Stemming': df_stemming,
           'Tokenization': df_token}
writer = pd.ExcelWriter('./Datap3p_Preprocessed.xlsx', engine='xlsxwriter', mode='w')

for data_sheet in dataAll.keys():
    dataAll[data_sheet].to_excel(writer, sheet_name=data_sheet, index=False)

writer.close()
writer.handles = None

print("\n\n\nData Preprocessed Saved!!!")

wacana panjang jabat jokowi presiden gaung isu tunda milu reda isu jokowi periode cuat isu panas asosiasi perintah desa indonesia apdesi pro konstitusi
fakta realita bantah presiden jokowi presiden taat konstitusi kali beliau patuh undang undang tolak periode jabat presiden ragu
kait seru jabat presiden periode jokowi tegas taat konstitusi konstitusi atur jabat presiden maksimal periode gaduh kait jabat periode paham
etika guna kaos jokowi periode etika undang undang jabat presiden maksimal periode
presiden jokowi tuju jabat presiden periode
presiden jokowi konsisten tolak tunda milu panjang jabat presiden presiden periode
presiden jokowi tuju usul jabat presiden periode
presiden jokowi tolak jabat wacana presiden periode langgar konstitusi milu laksana sesuai tanggal tetap
presiden jokowi tolak jabat periode mudik utama prokes
jokowi milu jabat presiden periode mudik utama prokes
presiden jokowi milu laksana nama tunda milu nama panjang jabat periode presiden jokowi taat konstitusi
kl

menteri polhukam mahfud angkat bicara kait isu panjang jabat presiden periode mahfud wacana presiden periode tutup buku mahfud sebut tunduk konstitusi jabat presiden maksimal periode
partai koalisi kelompok dukung jokowi juang panjang jabat presiden periode mpr rencana amandemen konstitusi gawat jokowi sayap
presiden jokowi merespon minat tambah jabat periode
menteri polhukam mahfud angkat bicara kait isu panjang jabat presiden periode mahfud wacana presiden periode tutup buku demokrasi
presiden jokowi tanggap jabat presiden periode
fakta presiden jokowi konsisten tolak periode paksa demo tolak jabat periode maksud selubung tangkap provokator rusuh mahasewa mahasiswa
aksi april mahasiswi teriak tuntut salah satu tolak wacana jabat presiden periode sehat jokowi bilang taat konstitusi
konsistensi jokowi tolak tunda milu jabat presiden periode mahasewa mahasiswa
raden ahy darah biru senang bahas jabat periode sby calon capres silah presiden tolak jabat presiden periode
jokowi ubah amandem

jokowi tolak wacana jabat presiden periode kpu mnyatakan milu ada demo tolak tunda milu panjang jabat presiden arti milik modal biaya kontrak demo
laman resmi milik dinas lembaga alat website pemkab trenggalek retas hacker retas ramai aksi tolak wacana tunda milu panjang jabat presiden
cerdas sumber provokator jabat jerumus langgar konstitusi wacana tunda milu jabat presiden periode
mahasiswa solo unjuk tolak panjang jabat presiden tanggap aksi wali kota solo anak presiden joko widodo gibran rakabuming sebut rencana jokowi jabat presiden periode
jokowi pundasi ungkit kuat bangun ekonomi bangsa sejahtera rakyat capai atas jokowi periode panjang jabat presiden ucap terimakasih anak bangsa
reaksi orang sby gerilya masif galang kepala desa gerilya parlemen reaksi keras mahasiswa banding lengkap aku kampus banding
presiden jokowi panjang jabat presiden milu feb bem demo april isu tolak periode wakil wakil bohirkah
tolak jabat presiden periode gibran bicara langsung jokowi
selesai benci joko

muda mahasiswa aksi april turun jokowi tolak amandemen undang undang dasar jabat presiden periode paksa turun lihat video deddy burhan saidi
presiden jokowi tolak panjang jabat ambil tindak agar menteri bebas hembus isu periode pecat rakyat konsisten omong
demo turun jokowi demo tolak panjang jabat presiden periode ide lahir luhut sejati demo untuk luhut jokowi
satu orang percaya tambah jabat presiden tunda milu periode singkir orang wacana lingkar
dukung demo tolak jabat presiden periode jokowi minat presiden periode berani ikut demo mana demo rusuh ceroboh
pecah demo akibat jokowi biar manuver luhut isu panjang jabat presiden periode
dikotomi tuju jabat presiden periode kubu kontra jokowi tuju pro jokowi aspirasi lihat lihat demo kelompok pro kontra jokowi
mahasiswa motone picek isu jabat presiden periode jokowi salah jokowi ajar
geram presiden jokowi amandemen tolak jabat presiden periode
isu lagipula jabat presiden periode sah jokowi maju lelah
wakil ketua dpr sufmi dasco ahmad par

partai politik culas dukung oposisi pks bohong suara tum golkar wacana periode dukung demo anti periode pks tolak panjang jabat presiden dukung gubernur
presiden serius pecat menteri jabat jabat kemarin ngeyel periode panjang jabat presiden
presiden rakyat percaya gaduh wacana periode jokowi jabat jokowi milik perilaku pribadi ganda
bicara presiden republik indonesia panjang jabat periode
manuver presiden periode panjang jabat presiden tanggung gaduh undur pecat
jabat presiden periode undang undang jokowi maju pilpres undang undang amandemen jokowi maju pilpres hendak rakyat jokowi periode dasar otak busuk
tinggal kader terima kasih presiden tuntut maaf terima kasih beliau tuduh jokowi balik isu periode panjang jabat
tuntut kader demokrat tum maaf terima kasih presiden tuduh beliau balik wacana periode panjang jabat presiden alau maaf mk doa demokrat hancur milu
panjang jabat rekomendasi milu keliru jokowi dukung presiden moga periode jadwal milu sepakat resmi masif struktur
presiden t

...

larang jajar perintah bahas periode jokowi singgung keras netizen larang diam diam presiden jokowi larang jajar perintah bahas periode panjang jabat tunda
hilang tolak wacana amandemen undang undang dasar tunda milu panjang jabat presiden usul jabat periode
ketiak wacana jabat presiden periode amandemen undang undang heboh surup demo tuntut turun iring caci hina otak buat anggap pintar sombong
periode panjang jabat presiden luka hati yo
tolak tunda milu presiden periode tolak turun presiden jokowi mari taat konstitusi presiden menteri henti wacana tunda milu panjang jabat mari demokrasi etika taat konstitusi
wakil ketua partai rakyat adil makmur prima alif kamal sambut nyata presiden joko widodo larang menteri bicara isu tunda milu panjang jabat presiden
jokowi menteri bicara tunda milu jabat periode rakyat jokowi mpr diam kesah tunda milu jabat presiden periode
instropeksi partai iibu demo karat dosa kemarin kuasa alami karma hancur tambah mua akibat pandang bodoh bahas tunda milu tam

jokowi periode sumpah allah masyarakat indonesia taat undang undang dasar jabat presiden periode undang undang dasar amandemen
tuju presiden jokowi pilih sesuai konstitusi jabat periode
orang orang lingkar joko widodo manuver wujud skenario tunda milu panjang jabat presiden bulat tekad jokowi periode deras
percaya presiden taat konstitusi tolak wacana periode tambah jabat
republik indonesia negerepublik indonesia mendagrepublik indonesia tito karnavian tagih anggota komisi ii dpr republik indonesia guspardi gaus wacana jabat presiden jokowi periode cuat
dpr kemendagri tegur apdesi dukung panjang jabat presiden jokowi periode
dengar jokowi periode bahasa jokowi turun jabat habis undang undang dasar alau presiden jabat periode
faldo presiden jokowi sikap panjang jabat presiden jokowi faldo taat atur konstitusi
jokowi rela tampar wacana jabat presiden periode kompas april pennyunting fitria chusna farisa
panjang jabat presiden periode pulau gadai utang
tanya jabat presiden periode preside

mendagri tito karnavia tanggap kabar apdesi dukung panjang jabat presiden
presiden joko widodo buka suara kait isu tambah jabat jokowi patuh atur tuang konstitusi
usul jabat presiden periode bicara musyawarah mufakat jabat presiden periode pimpin presiden hasil milu pilpres bela penting rezim jokowi kroni
harus presiden tunda milu presiden periode jabat beliau beliau bilang patuh konstitusi politik citra lebih konsistensi
tunda milu jokowi periode langkah roboh konstitusi undang undang dasar amandemen gus dur landas penting bangsa negara batas jabat presiden penting kuasa balik
presiden tanggap aspirasi masyarakat tambah jabat presiden periode tinjau candi borobudur kab magelang prov jawa rabu
coba runut berita target nafsu sukses periode panjang jabat mobilisasi kades dukung presiden periode hadir apdesi palsu jokowi lbp kena pidana sastrawan politik
maaf maaf panjang jabat presiden periode
pergi harga pertamax mahal pertalite stok poerwa turun harga bbm tolak presiden periode
tim rum

sebalikya jamin alau jokowi periode kawan inti langgar konstitusi undang undang negeri jabat presiden periode titik penuh duka sesuai atur konstitusi undang undang ubah atur
lanjut maslahat bangsa negara sesuai konstitusi ikut oknum jilat jilat keliling jabat presiden periode oknum periode cundang cari aman kkn
mas dpr ngubah undang undang dasar jabat presiden periode berat rakyat terima paham konstitusi kecuali kalimat atur jabat presiden periode inti
partai demokrat muncul wacana jokowi periode arti gera terorisme konstitusi
asosiasi perintah desa indonesia apdesi kabupaten tangerang dukung tambah jabat presiden joko widodo jokowi periode
wacana jabat presiden periode ahy tolak jokowi taat konstitusi
coba lihat menteri sibuk suara jabat presiden periode panjang ketua partai menteri dukung dukung jokowi narasi jokowi periode
wacana periode panjang jabat presiden komporsisi demokarat partai komunis sejati tuduh presiden terus
polemik periode jabat presiden jokowi taat konstitusi
presid

polemik jabat presiden periode presiden jokowi taat konstitusi
jokowi taat konstitusi jabat presiden periode negara demokrasi wajar alau teriak periode tunda milu jokowi milu
temu unjung candi borobudur keluarga tum partai demokrat agus harimurti yudhoyono tanggap isu wacana panjang jabat presiden joko widodo tolak
minta penuh suara salah orang dukung jokowi apresiasi kerja jabat presiden periode khianat reformasi
paksa gua bilang tunda mika jabat presiden periode undang undang draft rundang undang wacana wacana anies capres
apdesi suruh jokowi kepala desa jabat kepala desa periode klau presiden periode
ketua apdesi sah kemenkumham terima organisasi catut dukung jokowi periode apdesi dukung jokowi periode sikap taat konstitusi jabat presiden maksimal periode
pks khawatir kepala desa suara panjang jabat presiden jokowi
wacana panjang jabat presiden hangat
ketua dpp pks mardani ali sera nilai kepala desa jabat presiden jokowi panjang periode ajak jokowi jabat kepala desa
tenaga ahli utam

In [36]:
df_preprocess = pd.read_excel('./Datap3p_Preprocessed.xlsx', engine='openpyxl', sheet_name="Cleansing")
df_preprocess.drop_duplicates()
df_preprocess

,Created_at,Username,Tweets
0,2022-04-29 07:12:48,Anggita_lung,wacana perpanjangan masa jabatan jokowi sebaga...
1,2022-04-29 07:08:48,Penyair_Berdiri,fakta realita tak terbantahkan bahwa presiden ...
2,2022-04-29 07:03:00,garuda080,terkait seruan jabatan presiden tiga periode j...
3,2022-04-29 05:45:45,raenovaldy_,ya kalau ini tidak beretika berarti pengguna k...
4,2022-04-28 14:44:09,NadiaZzz90,presiden jokowi tidak setuju jabatan presiden ...
...,...,...,...
1289,2022-03-27 18:06:10,divamarsadhia,presiden jokowi tolak jabatan presiden periode
1290,2022-03-27 16:40:47,ollasuarsya,presiden jokowi tolak jabatan presiden periode
1291,2022-03-27 16:40:37,rudi_firnanda,presiden jokowi tolak jabatan presiden periode
1292,2022-03-27 14:13:40,I17082000,share disini berita atau video pak menginginka...


In [ ]:
labeldata = LabellingData()

# df_label = df_stemming.copy()
# df_label['Label'] = labeldata.Lexicon_based(list(df_label['Tweets']))
# df_label['Label'] = df_label['Label'].apply(labeldata.Analyze_score)

# new_df = pd.DataFrame({'Tweets':df_label["Tweets"],
#                    'Label':df_label['Label']})

df_label = pd.read_excel('./Datap3p_2variabel.xlsx', engine='openpyxl')

In [ ]:
# resultAnalysis
tweets_positive = [t for t in df_label["Label"] if t == "Positif"]
tweets_negative = [t for t in df_label["Label"] if t == "Negatif"]
tweets_netral = [t for t in df_label["Label"] if t == "Netral"]

print("Hasil Sentimen Analisis")
print("Positif : ", len(tweets_positive), "({}%)".format(100*len(tweets_positive)/len(df_label)))
print("Negatif : ", len(tweets_negative), "({}%)".format(100*len(tweets_negative)/len(df_label)))
print("Netral  : ", len(tweets_netral), "({}%)".format(100*len(tweets_netral)/len(df_label)))

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
import pandas as pd

df_label = pd.read_excel('./Datap3p_2variabel.xlsx', engine='openpyxl')

# TF-IDF
tf = TfidfVectorizer()
lb = LabelBinarizer()

text_tf = tf.fit_transform(df_label['Clean_Tweets']).toarray()
label_lb = lb.fit_transform(df_label['Label']).ravel()

X_train,X_test,y_train,y_test = train_test_split(text_tf, label_lb,test_size=0.2, random_state = 43)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

In [16]:
%%time
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import pickle

df_label = pd.read_excel('./Datap3p_clean.xlsx', engine='openpyxl', sheet_name='Stemming')

tweets_positive = [t for t in df_label["Label"] if t == "Positif"]
tweets_negative = [t for t in df_label["Label"] if t == "Negatif"]
tweets_netral = [t for t in df_label["Label"] if t == "Netral"]

print("Jumlah Seluruh Sentimen")
print("Positif : ", len(tweets_positive), "({}%)".format(100*len(tweets_positive)/len(df_label)))
print("Negatif : ", len(tweets_negative), "({}%)".format(100*len(tweets_negative)/len(df_label)))
print("Netral  : ", len(tweets_netral), "({}%)".format(100*len(tweets_netral)/len(df_label)))
print("-"*30)
X_train,X_test,y_train,y_test = train_test_split(df_label['Tweets'], df_label['Label'], test_size=0.1, random_state = 0, stratify=df_label['Label'])

X_train_positive = [t for t in y_train if t == "Positif"]
X_train_negative = [t for t in y_train if t == "Negatif"]
X_train_netral = [t for t in y_train if t == "Netral"]

print("Jumlah Sentimen X_train")
print("Positif : ", len(X_train_positive), "({}%)".format(100*len(X_train_positive)/len(X_train)))
print("Negatif : ", len(X_train_negative), "({}%)".format(100*len(X_train_negative)/len(X_train)))
print("Netral  : ", len(X_train_netral), "({}%)".format(100*len(X_train_netral)/len(X_train)))

print("-"*30)

X_test_positive = [t for t in y_test if t == "Positif"]
X_test_negative = [t for t in y_test if t == "Negatif"]
X_test_netral = [t for t in y_test if t == "Netral"]

print("Jumlah Sentimen X_test")
print("Positif : ", len(X_test_positive), "({}%)".format(100*len(X_test_positive)/len(X_test)))
print("Negatif : ", len(X_test_negative), "({}%)".format(100*len(X_test_negative)/len(X_test)))
print("Netral  : ", len(X_test_netral), "({}%)".format(100*len(X_test_netral)/len(X_test)))

print("="*30)

# TF-IDF
tf = TfidfVectorizer()
lb = LabelBinarizer()


label_lb = lb.fit_transform(df_label['Label']).ravel() 

X_train1,X_test1,y_train1,y_test1 = train_test_split(text_tf, df_label['Label'], test_size=0.1, random_state = 0, stratify=label_lb)
X_train,X_test,y_train,y_test = train_test_split(text_tf, label_lb, test_size=0.1, random_state = 0, stratify=label_lb)

# X_train,X_test,y_train,y_test = train_test_split(df_label['Clean_Tweets'], df_label['Label'], test_size=0.2, random_state = 0)

# X_train = tf.fit_transform(X_train).toarray()
# y_train = lb.fit_transform(y_train).ravel()

# X_test = tf.fit_transform(X_test).toarray()
# y_test = lb.fit_transform(y_test).ravel()

print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)
me = MultiNB()
me.fit(X_train, y_train)
    
predicted = me.predict(X_test)
# print(me.predict_proba)
print(predicted)


print("========================================================")
print(f'confusion matrix:\n {confusion_matrix(y_test, predicted)}')
print("========================================================\n")
print(classification_report(y_test, predicted, zero_division = 0))

Jumlah Seluruh Sentimen
Positif :  103 (50.0%)
Negatif :  103 (50.0%)
Netral  :  0 (0.0%)
------------------------------
Jumlah Sentimen X_train
Positif :  93 (50.270270270270274%)
Negatif :  92 (49.729729729729726%)
Netral  :  0 (0.0%)
------------------------------
Jumlah Sentimen X_test
Positif :  10 (47.61904761904762%)
Negatif :  11 (52.38095238095238%)
Netral  :  0 (0.0%)
(185, 1801) (21, 1801) (185,) (21,)


NameError: name 'MultiNB' is not defined

In [18]:
true_positif = 0
true_negatif = 0
false_positif = 0
false_negatif = 0

for i in range(len(predicted)):
    if predicted[i] == 1:
        if predicted[i] == y_test[i]:
            true_positif += 1
        else:
            false_negatif += 1
    else:
        if predicted[i] == y_test[i]:
            true_negatif += 1
        else:
            false_positif += 1

def Analyze_score(score):
    if score < 0 or score == 0:
        return 'Negatif'
    # elif score == 0:
    #    return "Netral"
    else:
        return "Positif"

predicted = list(predicted)
predicted = [Analyze_score(x) for x in predicted]

In [19]:
predicted

['Positif',
 'Negatif',
 'Negatif',
 'Positif',
 'Positif',
 'Positif',
 'Positif',
 'Negatif',
 'Positif',
 'Negatif',
 'Negatif',
 'Negatif',
 'Negatif',
 'Negatif',
 'Positif',
 'Positif',
 'Positif',
 'Negatif',
 'Positif',
 'Negatif',
 'Positif',
 'Negatif',
 'Negatif',
 'Positif',
 'Positif',
 'Positif',
 'Negatif',
 'Positif',
 'Positif',
 'Positif',
 'Positif',
 'Negatif',
 'Negatif',
 'Positif',
 'Positif',
 'Negatif',
 'Negatif',
 'Positif',
 'Positif',
 'Positif']

In [883]:
from flask import Flask
from flask_sqlalchemy import SQLAlchemy

app = Flask(__name__)
app.config['SQLALCHEMY_DATABASE_URI'] = 'mysql://root:''@localhost/ta_1811503083'
app.config['SQLALCHEMY_TRACK_MODIFICATIONS'] = False
app.config['SECRET_KEY'] = "b1258fcd01d43b2d6cb01ec8"

db = SQLAlchemy(app)

class Dataset(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    created_at = db.Column(db.String(100))
    username = db.Column(db.String(100))
    raw_tweets = db.Column(db.Text)
    clean_tweets = db.Column(db.Text)
    label = db.Column(db.String(10))
    
data_clean = Dataset.query.filter('presiden').all()

In [898]:
data = Dataset.query.all()
        
raw_headers = ['id', 'created_at', 'username', 'raw_tweets', 'clean_tweets']

data_raw=[]
for result in data:
    rs = [str(result.id), str(result.created_at), str(result.username), str(result.raw_tweets), str(result.clean_tweets)]
    data_raw.append(dict(zip(raw_headers,rs)))

if data_raw[0]['clean_tweets'] == 'None':
    print("true")
else:
    print("false")

OperationalError: (MySQLdb._exceptions.OperationalError) (2006, 'Server has gone away')
[SQL: SELECT dataset.id AS dataset_id, dataset.created_at AS dataset_created_at, dataset.username AS dataset_username, dataset.raw_tweets AS dataset_raw_tweets, dataset.clean_tweets AS dataset_clean_tweets, dataset.label AS dataset_label 
FROM dataset]
(Background on this error at: https://sqlalche.me/e/14/e3q8)

In [901]:
for i in range(0, 11):
    print(i-1)

-1
0
1
2
3
4
5
6
7
8
9


In [892]:
dataPositif = Dataset.query.filter_by(label='Positif').all()
dataNegatif = Dataset.query.filter_by(label='Negatif').all()
totalSplit = 0

if len(dataPositif) >= len(dataNegatif):
    if len(dataNegatif) % 2 == 0:
        totalSplit = len(dataNegatif)
    else:
        totalSplit = len(dataNegatif) - 1
else:
    if len(dataPositif) % 2 == 0:
        totalSplit = len(dataPositif)
    else:
        totalSplit = dataPositif - 1
        
print(int(totalSplit/2))

51


In [31]:
cl = {'created_at': [], 'raw_tweets': [],'clean_tweets': [], 'label': []}
num_p = 0
num_n = 0

for d in data_clean:
    if d.label == "Positif" and num_p < 200:
        cl['created_at'].append(d.created_at)
        cl['raw_tweets'].append(d.raw_tweets)
        cl['clean_tweets'].append(d.clean_tweets)
        cl['label'].append(d.label)
        num_p += 1
            
    if d.label == "Negatif" and num_n < 200:
        cl['created_at'].append(d.created_at)
        cl['raw_tweets'].append(d.raw_tweets)
        cl['clean_tweets'].append(d.clean_tweets)
        cl['label'].append(d.label)
        num_n += 1
        
tweets_positive = [t for t in cl["label"] if t == "Positif"]
tweets_negative = [t for t in cl["label"] if t == "Negatif"]
tweets_netral = [t for t in cl["label"] if t == "Netral"]

print("Jumlah Seluruh Sentimen")
print("Positif : ", len(tweets_positive), "({}%)".format(100*len(tweets_positive)/len(df_label)))
print("Negatif : ", len(tweets_negative), "({}%)".format(100*len(tweets_negative)/len(df_label)))
print("Netral  : ", len(tweets_netral), "({}%)".format(100*len(tweets_netral)/len(df_label)))

Jumlah Seluruh Sentimen
Positif :  200 (20.0%)
Negatif :  200 (20.0%)
Netral  :  0 (0.0%)


In [837]:
df_label = pd.read_excel('./Datap3p_206.xlsx', engine='openpyxl')

tweets_positive = [t for t in df_label["Label"] if t == "Positif"]
tweets_negative = [t for t in df_label["Label"] if t == "Negatif"]
tweets_netral = [t for t in df_label["Label"] if t == "Netral"]

print("Jumlah Seluruh Sentimen")
print("Positif : ", len(tweets_positive), "({}%)".format(100*len(tweets_positive)/len(df_label)))
print("Negatif : ", len(tweets_negative), "({}%)".format(100*len(tweets_negative)/len(df_label)))
print("Netral  : ", len(tweets_netral), "({}%)".format(100*len(tweets_netral)/len(df_label)))
print("-"*30)

X_train,X_test,y_train,y_test = train_test_split(df_label['Tweets'], df_label['Label'], test_size=0.1, random_state = 0, stratify=df_label['Label'])

X_train_positive = [t for t in y_train if t == "Positif"]
X_train_negative = [t for t in y_train if t == "Negatif"]
X_train_netral = [t for t in y_train if t == "Netral"]

print("Jumlah Sentimen X_train")
print("Positif : ", len(X_train_positive), "({}%)".format(100*len(X_train_positive)/len(X_train)))
print("Negatif : ", len(X_train_negative), "({}%)".format(100*len(X_train_negative)/len(X_train)))
print("Netral  : ", len(X_train_netral), "({}%)".format(100*len(X_train_netral)/len(X_train)))

print("-"*30)

X_test_positive = [t for t in y_test if t == "Positif"]
X_test_negative = [t for t in y_test if t == "Negatif"]
X_test_netral = [t for t in y_test if t == "Netral"]

print("Jumlah Sentimen X_test")
print("Positif : ", len(X_test_positive), "({}%)".format(100*len(X_test_positive)/len(X_test)))
print("Negatif : ", len(X_test_negative), "({}%)".format(100*len(X_test_negative)/len(X_test)))
print("Netral  : ", len(X_test_netral), "({}%)".format(100*len(X_test_netral)/len(X_test)))

Jumlah Seluruh Sentimen
Positif :  103 (50.0%)
Negatif :  103 (50.0%)
Netral  :  0 (0.0%)
------------------------------
Jumlah Sentimen X_train
Positif :  93 (50.270270270270274%)
Negatif :  92 (49.729729729729726%)
Netral  :  0 (0.0%)
------------------------------
Jumlah Sentimen X_test
Positif :  10 (47.61904761904762%)
Negatif :  11 (52.38095238095238%)
Netral  :  0 (0.0%)


In [ ]:
X_test

In [45]:
def Analyze_score(score):
        if score < 0 or score == 0:
            return 'Negatif'
        # elif score == 0:
        #    return "Netral"
        else:
            return "Positif"

datapredict = list(yhat)
datapredict = [Analyze_score(x) for x in datapredict]
print(datapredict)

Negatif


In [ ]:
np.log(me.class_priors_)

In [61]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np

df_label = pd.read_excel('./Datap3p.xlsx', engine='openpyxl')

# TF-IDF
tf = TfidfVectorizer()
lb = LabelBinarizer()

X_train,X_test,y_train,y_test = train_test_split(tf.fit_transform(df_label['Clean Tweets']), lb.fit_transform(df_label['Label']).ravel(),test_size=0.2, random_state = 23)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

y = y_train
n_samples, n_features = X_train.shape
classes_ = np.unique(y_train)
n_classes_ = classes_.shape[0]

(800, 1832) (200, 1832) (800,) (200,)


In [ ]:
print(X_train)

In [ ]:
uniques = []
for i in range(n_features):
    tmp = np.unique(X_train[:,i])
    uniques.append( tmp )
    
uniques

In [ ]:
N_yi = np.zeros((n_classes_, n_features)) # feature count
N_y = np.zeros((n_classes_)) # total count 

print(N_yi)

In [ ]:
for i in classes_: # x axis
    indices = np.argwhere(y==i).flatten()
    columnwise_sum = []
    for j in range(n_features): # y axis
        columnwise_sum.append(np.sum(X_train[indices,j]))

    N_yi[i] = columnwise_sum # 2d
    N_y[i] = np.sum(columnwise_sum) # 1d

In [ ]:
for y in N_y:
    print(y)

In [ ]:
for yi in N_yi:
    for x in range(len(yi)):
        print(yi[x])

In [ ]:
c = X_train.tocoo()  # convert the sparse matrix to COO format
df = pd.DataFrame({'node1': c.row, 'node2': c.col, 'edge_weight': c.data})
df

In [14]:
# Performa Algoritma Naive Bayes
from sklearn.naive_bayes import MultinomialNB, GaussianNB, BernoulliNB
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.utils.validation import check_X_y, check_array
import pandas as pd
import numpy as np

df_label = pd.read_excel('./Datap3p_clean.xlsx', engine='openpyxl')

# TF-IDF
tf = TfidfVectorizer()
lb = LabelBinarizer()

# text_tf = tf.fit_transform(df_label['Clean Tweets'])
# label_lb = lb.fit_transform(df_label['Label'])

# X_train,X_test,y_train,y_test = train_test_split(text_tf, df_label['Label'],test_size=0.2, random_state = 0)

text_tf = tf.fit_transform(df_label['Tweets']).toarray()
label_lb = lb.fit_transform(df_label['Label']).ravel()

X_train,X_test,y_train,y_test = train_test_split(text_tf, label_lb, test_size=0.1, random_state = 0, stratify=label_lb)
# X_train,X_test,y_train,y_test = train_test_split(text_tf, label_lb, test_size=0.2, random_state = 0, stratify=label_lb)

print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

# clf = MultinomialNB().fit(X_train, y_train)
# predicted = clf.predict(X_test)

me = NaiveBayes()
me = me.fit(X_train, y_train)
predicted = me.predict(X_test)


print("========================================================")
print(predicted)
# print(me.predict_proba)
print("========================================================\n")

# clf = GaussianNB().fit(X_train.toarray(), y_train)
# predicted = clf.predict(X_test.toarray())
print("MultinomialNB Accuracy : ", accuracy_score(y_test, predicted))
print("MultinomialNB Precision : ", precision_score(y_test, predicted, average = "micro", pos_label = "Negatif"))
print("MultinomialNB Recall : ", recall_score(y_test, predicted, average = "micro", pos_label = "Negatif"))
print("MultinomialNB f1_score : ", f1_score(y_test, predicted, average = "micro", pos_label = "Negatif"))

print(f'confusion matrix:\n {confusion_matrix(y_test, predicted)}')
print("========================================================\n")
print(classification_report(y_test, predicted, zero_division = 0))

(185, 1801) (21, 1801) (185,) (21,)
[1 0 0 1 1 0 0 1 1 1 1 0 0 0 1 1 1 0 1 1 1]

MultinomialNB Accuracy :  0.8571428571428571
MultinomialNB Precision :  0.8571428571428571
MultinomialNB Recall :  0.8571428571428571
MultinomialNB f1_score :  0.8571428571428571
confusion matrix:
 [[ 8  3]
 [ 0 10]]

              precision    recall  f1-score   support

           0       1.00      0.73      0.84        11
           1       0.77      1.00      0.87        10

    accuracy                           0.86        21
   macro avg       0.88      0.86      0.86        21
weighted avg       0.89      0.86      0.86        21



C:\Users\user\AppData\Local\Temp/ipykernel_6268/1977132169.py:17: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  x_per_class = np.array([x[y == c] for c in np.unique(y)])
C:\Users\user\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1298: UserWarning: Note that pos_label (set to 'Negatif') is ignored when average != 'binary' (got 'micro'). You may use labels=[pos_label] to specify a single positive class.
  warnings.warn("Note that pos_label (set to %r) is ignored when "
C:\Users\user\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1298: UserWarning: Note that pos_label (set to 'Negatif') is ignored when average != 'binary' (got 'micro'). You may use labels=[pos_label] to specify a single positive class.
  warnings.warn("Note that pos_labe

In [5]:
class NaiveBayes():
    def __init__(self, alpha=1.0):
        self.prior = None
        self.word_counts = None
        self.lk_word = None
        self.alpha = alpha
        
    def fit(self, x, y):
        '''
        Fit the features and the labels
        Calculate prior, word_counts and lk_word
        '''
        x, y = check_X_y(x, y)
        n = x.shape[0]
        
        # calculate the prior - number of text belonging to a particular class (real or fake)
        x_per_class = np.array([x[y == c] for c in np.unique(y)])
        self.prior = np.array([len(x_class) / n for x_class in x_per_class])
        
        # calculate the likelihood for each word 'lk_word'
        self.word_counts = np.array([sub_arr.sum(axis=0) for sub_arr in x_per_class]) + self.alpha
        self.lk_word = self.word_counts / self.word_counts.sum(axis=1).reshape(-1, 1)
        
        return self
    
    def _get_class_numerators(self, x):
        '''
        Calculate for each class, the likelihood that an entire message conditional
        on the message belonging to a particular class (real or fake)
        '''
        n, m = x.shape[0], self.prior.shape[0]
        
        class_numerators = np.zeros(shape=(n, m))
        for i, word in enumerate(x):
            word_exists = word.astype(bool)
            lk_words_present = self.lk_word[:, word_exists] ** word[word_exists]
            lk_message = (lk_words_present).prod(axis=1)
            class_numerators[i] = lk_message * self.prior
        
        return class_numerators
    
    def _normalized_conditional_probs(self, class_numerators):
        '''
        Conditional probabilities = class_numerators / normalize_term
        '''
        # normalize term is the likelihood of an entire message (addition of all words in a row)
        normalize_term = class_numerators.sum(axis=1).reshape(-1,1)
        conditional_probs = class_numerators / normalize_term
        assert(conditional_probs.sum(axis=1) - 1 < 0.001).all(), 'rows should sum to 1'
        
        return conditional_probs
    
    def predict_proba(self, x):
        '''
        Return the probabilities for each class (fake or real)
        '''
        class_numerators = self._get_class_numerators(x)
        conditional_probs = self._normalized_conditional_probs(class_numerators)
        
        return conditional_probs
    

    def predict(self, x):
        '''
        Return the answer with the highest probability (argmax)
        '''
        return self.predict_proba(x).argmax(axis=1)

# ============================================================== #

# Source Code Lama Part 2

In [ ]:
import tweepy
import pandas as pd
import datetime
import string

auth = tweepy.OAuth1UserHandler(
    consumer_key = 'mAViUQaittIvlfAg99yY3yaSA',
    consumer_secret = 'SabL3v6MIsxkSTUTC4c1bhqodbss4pkrIJYz9dXld6txZVH0BL',
    access_token = '1273402051817467904-VwNfdVmhlTWIyRFzIKzUAq1UX7CzoA',
    access_token_secret = 'ipMnMqhoSFNKzTU1dTeJOIkfLKalkfJumClH9CqiB0mFL'
)

# Inisialisasi API untuk akses twitter RESTful API methods
api = tweepy.API(auth)

keyword = ['hepatitis']

tanggal_awal = '2022-05-26'
tanggal_akhir = '2022-05-27'
new_tweets = api.search_tweets(q = ', '.join(keyword) + ' -filter:retweets', until = '2022-05-27', result_type='recent', lang = 'id', tweet_mode = 'extended')

startDate = datetime.datetime.strptime(tanggal_awal, '%Y-%m-%d')
endDate = datetime.datetime.strptime(tanggal_akhir, '%Y-%m-%d')
endDate =  endDate.replace(hour=23, minute=59, second=59)

In [ ]:
startDate_timestamp =  datetime.datetime.strftime(startDate, '%Y-%m-%d')
endDate_timestamp2 =  datetime.datetime.strftime(endDate, '%Y-%m-%d')
final_timestamp2

In [ ]:
for t in new_tweets:
#     tw = str(t.created_at.date())
#     if str(t.created_at.date()) >= final_timestamp2:
#         print('ok')
    print(t.created_at.date())


In [ ]:
import tweepy
import pandas as pd
from datetime import datetime, timedelta
import string

# Inisialisasi autentikasi tweepy
auth = tweepy.OAuth1UserHandler(
    consumer_key = 'mAViUQaittIvlfAg99yY3yaSA',
    consumer_secret = 'SabL3v6MIsxkSTUTC4c1bhqodbss4pkrIJYz9dXld6txZVH0BL',
    access_token = '1273402051817467904-VwNfdVmhlTWIyRFzIKzUAq1UX7CzoA',
    access_token_secret = 'ipMnMqhoSFNKzTU1dTeJOIkfLKalkfJumClH9CqiB0mFL'
)

# Inisialisasi API untuk akses twitter RESTful API methods
api = tweepy.API(auth)

# Kata kunci pengambilan data twitter
keyword = ['hepatitis']

# Jumlah tweet yang ingin diambil
num_tweets = 100000

# List untuk menampung waktu dibuatnya tweet, username tweet dan text tweets
tweets_id = []
time_list = []
username_list = []
tweet_list = []

# List untuk menampung Id tiap tweet
tweet_id = []

last_id = 0 

while len(tweet_list) < num_tweets:
    try:
        # Pengambilan tweet dengan operator -filter:retweets untuk menyaring data retweets
        # paramter count, untuk mengambil data sebanyak 1000 tweets
        # paramater max_id untuk mencegah data redundan dengan mengambil id terkecil setelah fetching data twitter
        # paramter lang='id' untuk mengambil bahasa twitter
        # paramter tweet_mode='extended' untuk mengambil text tweet secara penuh
        new_tweets = api.search_tweets(q = ', '.join(keyword) + ' -filter:retweets', count = num_tweets, max_id = last_id - 1, lang = 'id', tweet_mode = 'extended')
    except tweepy.errors.TweepyException as e:
        # Menghentikan proses pengambilan tweet jika terjadi error
        print(f"Error : {e}")
        break
    else:
        # Menghentikan proses pengambilan tweet jika tidak ada lagi tweet yang relevan dengan keyword
        if not new_tweets:
            print(f"Could not find any more tweets!, total tweets : {len(tweet_list)}")
            break
        else:
            for tweet in new_tweets:
                # Hanya mengambil data dengan follower di atas 30 untuk mencegah akun bot
#                 if tweet.user.followers_count > 30:

                    # Menyimpan format waktu ke dalam tuple
                    struct_time = datetime.strptime(
                        f'{tweet.created_at.date()}' + ' '+ f'{tweet.created_at.time()}',
                        '%Y-%m-%d %H:%M:%S')

                    # Konversi waktu UTC +00:00 ke UTC +07:00 (Indonesia bagian barat)
                    offset_hours = 7
                    local_time = struct_time + timedelta(hours=offset_hours)

                    # Mengubah struktur waktu menjadi format 24 jam (0 - 23)
                    final_timestamp =  datetime.strftime(local_time, '%Y-%m-%d %H:%M:%S')

                    # Menyimpan semua data pada tiap list
                    tweets_id.append(str(tweet.id))
                    time_list.append(final_timestamp)
                    username_list.append(tweet.user.screen_name)
                    tweet_list.append(tweet.full_text)

                    # Menyimpan tiap Id tweets
                    tweet_id.append(tweet.id)
    
    # Update Id dengan mengambil Id terkecil
    last_id = min(tweet_id)

# Membuat struktur data 2 dimensi (kolom:baris)
df = pd.DataFrame({'Tweets Id': tweets_id,
                   'Created_at':time_list,
                   'Username':username_list,
                   'Tweets':tweet_list})

# Menyimpan data dalam format xlsx
# df.to_excel('./DataHep.xlsx', sheet_name='Raw Data', index=False)
print('Total data :', len(df))

In [829]:
import pandas as pd

readExcel = pd.read_excel('./Datap3p_206.xlsx')
print('Total data :', len(readExcel))
readExcel.head(10)

Total data : 206


,Created_at,Username,Tweets,Label
0,2022-04-25 13:54:01,Darmawan1076,"Walaupun saya gak setuju Presiden 3 periode, t...",Negatif
1,2022-04-24 08:04:00,garuda080,Presiden @jokowi menegaskan tidak ada penundaa...,Positif
2,2022-04-24 00:31:40,BajakSawah22,@cakimiNOW @TsamaraDKI @DivHumas_Polri Bapak k...,Negatif
3,2022-04-23 09:04:46,EriekaAmelia,Tolak 3 Periode\nTolak perpanjang masa jabatan...,Negatif
4,2022-04-23 07:39:15,ichsan_itcomm,@Kamajay35170591 @SerautPenaHati @HolidayGtr @...,Positif
5,2022-04-23 04:44:41,Fossil2022T,@FR33D0ML4ND 2024 sudah dikuasai oleh geng oli...,Negatif
6,2022-04-23 04:30:05,June_dya77,"Presiden @jokowi Tunduk Konstitusi, Jabatan Pr...",Positif
7,2022-04-22 17:16:12,PartaiSocmed,Masalahnya mulai dari partai2 koalisi hingga g...,Negatif
8,2022-04-22 09:40:07,KabarIndonesi11,Faktanya Presiden Jokowi konsisten menolak 3 p...,Negatif
9,2022-04-22 00:16:04,Kimberley20101,Jokowi masih nafsu untuk maju lagi 3 Periode. ...,Negatif


In [830]:
# Case Folding
def CaseFolding(tweetsText):
    tweetsText = tweetsText.lower()
    return tweetsText

df_casefolding = readExcel.copy()
df_casefolding['Tweets'] = df_casefolding['Tweets'].apply(CaseFolding)
print('Total data :', len(df_casefolding))
df_casefolding

Total data : 206


,Created_at,Username,Tweets,Label
0,2022-04-25 13:54:01,Darmawan1076,"walaupun saya gak setuju presiden 3 periode, t...",Negatif
1,2022-04-24 08:04:00,garuda080,presiden @jokowi menegaskan tidak ada penundaa...,Positif
2,2022-04-24 00:31:40,BajakSawah22,@cakiminow @tsamaradki @divhumas_polri bapak k...,Negatif
3,2022-04-23 09:04:46,EriekaAmelia,tolak 3 periode\ntolak perpanjang masa jabatan...,Negatif
4,2022-04-23 07:39:15,ichsan_itcomm,@kamajay35170591 @serautpenahati @holidaygtr @...,Positif
...,...,...,...,...
201,2022-03-30 13:17:37,tempodotco,partai demokrat mengkritik munculnya dukungan ...,Negatif
202,2022-03-30 12:44:37,harunst91,apdesi dukung jokowi 3 periode... memang nya k...,Negatif
203,2022-03-30 11:50:58,VicJerz,"nih orang memang provokatif sejati, semua org ...",Negatif
204,2022-03-27 14:13:40,I17082000,@rezim_tiktok @adiiiinnnnnn @rajamandiri70 @do...,Negatif


In [831]:
# Cleansing
import re

def Cleansing(tweetsText):
    # === Menghilangkan Tautan ===
    tweetsText = re.sub("\w+:\/\/\S+"," ", tweetsText)
    
    # === Menghapus karakter selain alfabet ===
    
    # Menghilangkan mention, hashtag, character reference
    tweetsText = re.sub('[@#&][A-Za-z0-9_]+'," ", tweetsText)
    
    # Menghilangkan tanda baca
    tweetsText = re.sub('[()!?;,]', ' ', tweetsText)
    tweetsText = re.sub('\[.*?\]',' ', tweetsText)

    # Menghilangkan tanda selain huruf
    tweetsText = re.sub("[^a-z]"," ", tweetsText)
    
    # === Menghapus 1 karakter ===
    tweetsText = re.sub(r"\b[a-z]\b", "", tweetsText)
    
    # Menghilangkan spasi lebih dari 1
    tweetsText = ' '.join(tweetsText.split())
    return tweetsText

df_cleansing = df_casefolding.copy()
df_cleansing['Tweets'] = df_cleansing['Tweets'].apply(Cleansing)

#Menghilangkan duplikat data
# df_cleansing = df_cleansing.drop_duplicates(subset='Tweets')

print('Total data :', len(df_cleansing))
df_cleansing

Total data : 206


,Created_at,Username,Tweets,Label
0,2022-04-25 13:54:01,Darmawan1076,walaupun saya gak setuju presiden periode tapi...,Negatif
1,2022-04-24 08:04:00,garuda080,presiden menegaskan tidak ada penundaan pemilu...,Positif
2,2022-04-24 00:31:40,BajakSawah22,bapak kan yg ajukan masa jabatan presiden supa...,Negatif
3,2022-04-23 09:04:46,EriekaAmelia,tolak periode tolak perpanjang masa jabatan pr...,Negatif
4,2022-04-23 07:39:15,ichsan_itcomm,rakyat milih periode belajar lagi sono soal uu...,Positif
...,...,...,...,...
201,2022-03-30 13:17:37,tempodotco,partai demokrat mengkritik munculnya dukungan ...,Negatif
202,2022-03-30 12:44:37,harunst91,apdesi dukung jokowi periode memang nya kepala...,Negatif
203,2022-03-30 11:50:58,VicJerz,nih orang memang provokatif sejati semua org p...,Negatif
204,2022-03-27 14:13:40,I17082000,share disini berita atau video pak menginginka...,Negatif


In [832]:
# Replace Slangwords (Normalisasi)
def Slangwords(tweetsText):
    slang_file = open('../kamus/daftar_slangwords.txt', 'r')
    for line in slang_file:
        slangword = line.strip().split(":")
        if slangword[0] in tweetsText.split(" "):
            tweetsText = tweetsText.replace(slangword[0] + " ", slangword[1] + " ")
            tweetsText = tweetsText.replace(" " + slangword[0], " " + slangword[1])
    return tweetsText


df_slangwords = df_cleansing.copy()
df_slangwords['Tweets'] = df_slangwords['Tweets'].apply(Slangwords)
df_slangwords

,Created_at,Username,Tweets,Label
0,2022-04-25 13:54:01,Darmawan1076,walaupun saya tidak setuju presiden periode ta...,Negatif
1,2022-04-24 08:04:00,garuda080,presiden menegaskan tidak ada penundaan pemilu...,Positif
2,2022-04-24 00:31:40,BajakSawah22,bapak kan yang ajukan masa jabatan presiden su...,Negatif
3,2022-04-23 09:04:46,EriekaAmelia,tolak periode tolak perpanjang masa jabatan pr...,Negatif
4,2022-04-23 07:39:15,ichsan_itcomm,rakyat pilih periode belajar lagi sana soal uu...,Positif
...,...,...,...,...
201,2022-03-30 13:17:37,tempodotco,partai demokrat mengkritik munculnya dukungan ...,Negatif
202,2022-03-30 12:44:37,harunst91,apdesi dukung jokowi periode memang nya kepala...,Negatif
203,2022-03-30 11:50:58,VicJerz,nih orang memang provokatif sejati semua orang...,Negatif
204,2022-03-27 14:13:40,I17082000,bagi disini berita atau video pak menginginkan...,Negatif


In [833]:
# Stopwords Removal
def StopwordsRemoval(tweetsText):
    stopwords_file = open('../kamus/daftar_stopwords.txt', 'r')
    stopwords_list = list(map(lambda x:x.strip(),list(stopwords_file)))
    tweetsText = tweetsText.split()
    tweetsText = [w for w in tweetsText if not w in stopwords_list]
    tweetsText = " ".join(word for word in tweetsText)
    return tweetsText

df_stopwords_removal = df_slangwords.copy()
df_stopwords_removal['Tweets'] = [StopwordsRemoval(tw) for tw in df_stopwords_removal['Tweets']]
df_stopwords_removal

,Created_at,Username,Tweets,Label
0,2022-04-25 13:54:01,Darmawan1076,setuju presiden periode suka narasi sebelah te...,Negatif
1,2022-04-24 08:04:00,garuda080,presiden penundaan pemilu perpanjangan jabatan...,Positif
2,2022-04-24 00:31:40,BajakSawah22,ajukan jabatan presiden periode pemilu tunda m...,Negatif
3,2022-04-23 09:04:46,EriekaAmelia,tolak periode tolak perpanjang jabatan preside...,Negatif
4,2022-04-23 07:39:15,ichsan_itcomm,rakyat pilih periode belajar uu jabatan presid...,Positif
...,...,...,...,...
201,2022-03-30 13:17:37,tempodotco,partai demokrat mengkritik munculnya dukungan ...,Negatif
202,2022-03-30 12:44:37,harunst91,apdesi dukung jokowi periode kepala desa perio...,Negatif
203,2022-03-30 11:50:58,VicJerz,orang provokatif sejati orang pks buta tuli me...,Negatif
204,2022-03-27 14:13:40,I17082000,berita video jabatan presiden periode bicara f...,Negatif


In [ ]:
# %%time
# import swifter

# # Stemming
# from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

# def Stemming(tweetsText):
#     factory = StemmerFactory()
#     stemmer = factory.create_stemmer()
#     do = []
#     for w in tweetsText:
#         dt = stemmer.stem(w)
#         do.append(dt)
#     d_clean = []
#     d_clean = " ".join(do)
#     print(d_clean)
#     return d_clean

# df_stemming = df_token.copy()
# df_stemming['Tweets'] = df_stemming['Tweets'].swifter.apply(Stemming)
# df_stemming.head(10)

In [872]:
%%time

# Stemming
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

# except_words = ['pemilu', 'pertamax', 'setuju', 'luhut', 
#                 'airlangga', 'ketum', 'mulai', 'tini', 
#                 'kumparan', 'layanan', 'telan']

except_words = []

def Stemming(tweetsText):
    factory = StemmerFactory()
    stemmer = factory.create_stemmer()
    do = []
    for w in tweetsText.split():
        if w in except_words:
            do.append(w)
        else:
            dt = stemmer.stem(w)
            do.append(dt)
    d_clean = []
    d_clean = " ".join(do)
    print(d_clean)
    return d_clean

df_stemming = df_stopwords_removal.copy()
df_stemming['Tweets'] = df_stemming['Tweets'].apply(Stemming)
df_stemming.head(10)

tuju presiden periode suka narasi belah teriak turun jokowi kubu teriak panjang jabat jokowi
presiden tunda milu panjang jabat presiden periode henti gaduh laksana milu pilkada serentak jadwal tetap perintah dpr kpu bawaslu
aju jabat presiden periode milu tunda masuk jokowi perangkap
tolak periode tolak panjang jabat presiden turun harga bahan pokok turun harga bbm minyak bahas tuntas langgar ham tolak ruu pro oligarki cabut uu omnibuslaw jokowi gagal jokowi mundur
rakyat pilih periode ajar uu jabat presiden maksimal periode paksa langgar uu laku kriminal adil adil
asai kelompok oligarki jokowi henti bubar dpr mpr libat bicara periode amandemen uud panjang jabat presiden
presiden tunduk konstitusi jabat presiden maksimal periode mahfud wacana presiden periode tutup buku
partai koalisi kelompok dukung jokowi juang panjang jabat presiden periode mpr rencana amandemen konstitusi gawat jokowi sayap
fakta presiden jokowi konsisten tolak periode paksa demo tolak jabat periode maksud selubung

presiden jokowi taat konstitusi sesuai nyata beliau kali panjang jabat periode oceh isu libat perintah
mantan jubir jokowi tegak lurus reformasi beliau angkat jubir diskus langsung beliau jokowi setia jalan amanat reformasi kait batas jabat presiden periode
isu jabat periode topeng target utama bohir presiden jokowi lengser
tuju menteri amandemen uud tabu jabat presiden batas alat ukur demokrasi periode
presiden joko widodo jokowi wiranto kali ban isu panjang jabat tunda milu hitung kepala negara tolak isu
demo tolak panjang jabat presiden presiden periode eksekutif perintah jokowi salah alamat wenang presiden lembaga legislatif pantas nafsu politikus busuk kaum khilafah
sobat omong presiden kenal pegang pagi tempe sore kedelai omong usul jabat presiden panjang cari muka ubah wacana presiden periode demokratis perintah henti wacana presiden periode percaya
bem nusantara wiranto dialog isu wacana periode jabat presiden jokowi
arah luhut mul saran masyarakat
kepala staf presiden ksp resm

foreder apresiasi langkah presiden jokowi menteri henti bahas tunda jabat periode
presiden masuk jebak batman tunda panjang isu periode kawal sampai tuntas jabat sesuai atur undang laku
pecat jokowi tuju tunda milu periode jabat presiden bukti lbp kepala desa acara
maksud jokowi panjang jabat presiden wacana langsung eksekusi alias realisasi wacana rapat ketuk palu sah periode
presiden langgar konstitusi panjang jabat periode solusi mari turun presiden tepok dahi
henti wacana tunda milu sidang kabinet paripurna istana negara jokowi beliau tolak wacana presiden periode tunda milu pengaruh isu sumber orang hilang jabat kuasa
kadrun susah paham bahasa indonesia presiden jokowi ingat menteri suara tunda milu jabat presiden periode kadrun bahas
kadrun lengser jokowi hasut mahasiswa konten palsu fakta presiden jokowi jabat periode
tuju presiden jokowi pilih sesuai konstitusi jabat periode
orang orang lingkar joko widodo manuver wujud skenario tunda milu panjang jabat presiden bulat tekad jok

,Created_at,Username,Tweets,Label
0,2022-04-25 13:54:01,Darmawan1076,tuju presiden periode suka narasi belah teriak...,Negatif
1,2022-04-24 08:04:00,garuda080,presiden tunda milu panjang jabat presiden per...,Positif
2,2022-04-24 00:31:40,BajakSawah22,aju jabat presiden periode milu tunda masuk jo...,Negatif
3,2022-04-23 09:04:46,EriekaAmelia,tolak periode tolak panjang jabat presiden tur...,Negatif
4,2022-04-23 07:39:15,ichsan_itcomm,rakyat pilih periode ajar uu jabat presiden ma...,Positif
5,2022-04-23 04:44:41,Fossil2022T,asai kelompok oligarki jokowi henti bubar dpr ...,Negatif
6,2022-04-23 04:30:05,June_dya77,presiden tunduk konstitusi jabat presiden maks...,Positif
7,2022-04-22 17:16:12,PartaiSocmed,partai koalisi kelompok dukung jokowi juang pa...,Negatif
8,2022-04-22 09:40:07,KabarIndonesi11,fakta presiden jokowi konsisten tolak periode ...,Negatif
9,2022-04-22 00:16:04,Kimberley20101,jokowi nafsu maju periode panjang jabat tunda ...,Negatif


In [14]:
# except_words = ['pemilu', 'pertamax', 'setuju']
# teks = "saya akan ikut pemilu"
# for t in teks.split():
#     if t in except_words:
#         continue
#         print('ada')

In [873]:
# Tokenization
def Tokenization(tweetsText):
    tweetsText = tweetsText.split()
    return tweetsText

def Tokenization2(comments):
    nstr = comments.split(' ')
    dat = []
    a = -1
    for hu in nstr:
        a = a + 1
    if hu == '':
        dat.append(a)
    p = 0
    b = 0
    for q in dat:
        b = q - p
        del nstr[b]
        p = p + 1
    return nstr

df_token = df_stemming.copy()
df_token['Tweets'] = df_token['Tweets'].apply(Tokenization)
df_token.head(10)

,Created_at,Username,Tweets,Label
0,2022-04-25 13:54:01,Darmawan1076,"[tuju, presiden, periode, suka, narasi, belah,...",Negatif
1,2022-04-24 08:04:00,garuda080,"[presiden, tunda, milu, panjang, jabat, presid...",Positif
2,2022-04-24 00:31:40,BajakSawah22,"[aju, jabat, presiden, periode, milu, tunda, m...",Negatif
3,2022-04-23 09:04:46,EriekaAmelia,"[tolak, periode, tolak, panjang, jabat, presid...",Negatif
4,2022-04-23 07:39:15,ichsan_itcomm,"[rakyat, pilih, periode, ajar, uu, jabat, pres...",Positif
5,2022-04-23 04:44:41,Fossil2022T,"[asai, kelompok, oligarki, jokowi, henti, buba...",Negatif
6,2022-04-23 04:30:05,June_dya77,"[presiden, tunduk, konstitusi, jabat, presiden...",Positif
7,2022-04-22 17:16:12,PartaiSocmed,"[partai, koalisi, kelompok, dukung, jokowi, ju...",Negatif
8,2022-04-22 09:40:07,KabarIndonesi11,"[fakta, presiden, jokowi, konsisten, tolak, pe...",Negatif
9,2022-04-22 00:16:04,Kimberley20101,"[jokowi, nafsu, maju, periode, panjang, jabat,...",Negatif


In [874]:
dataAll = {'Raw Data': readExcel, 'Case Folding': df_casefolding, 'Cleansing': df_cleansing,
           'Normalization': df_slangwords, 'StopWords Removal': df_stopwords_removal, 'Stemming': df_stemming,
           'Tokenization': df_token}
writer = pd.ExcelWriter('./Datap3p_clean.xlsx', engine='xlsxwriter', mode='w')

for data_sheet in dataAll.keys():
    dataAll[data_sheet].to_excel(writer, sheet_name=data_sheet, index=False)

writer.close()
writer.handles = None

In [ ]:
%%time

# labelling
def Labelling(tweet_list):
    pos_list= open("./kamus/kata_positif.txt","r")
    pos_kata = pos_list.readlines()
    neg_list= open("./kamus/kata_negatif.txt","r")
    neg_kata = neg_list.readlines()  

    sentiments = [] #Membuat sebuah list menyimpan nilai sentiment

    #fungsi menghitung sentiment setiap tweet
    for item in tweet_list:
        print(item.strip())
        tweets = item.strip().split() #tokenization
        # print(tweets)

        count_p = 0 #nilai positif
        count_n = 0 #nilai negatif

        for tweet in tweets:
            for kata_pos in pos_kata:
                if kata_pos.strip().lower() == tweet.lower():
                    print(kata_pos, ['positif'])
                    count_p += 1
            for kata_neg in neg_kata:
                if kata_neg.strip().lower() == tweet.lower():
                    print(kata_neg, ['negatif'])
                    count_n += 1

        print ("positif: "+str(count_p))
        print ("negatif: "+str(count_n))
        sentiments.append(count_p - count_n)
    
    return sentiments

def analyze(score):
    if score < 0:
        return 'Negatif'
    elif score == 0:
        return "Netral"
    else:
        return "Positif"

df_label = df_stemming.copy()
df_label['Label'] = Labelling(list(df_label['Tweets']))
df_label['Label'] = df_label['Label'].apply(analyze)

new_df = pd.DataFrame({'Tweets':df_label["Tweets"],
                   'Label':df_label['Label']})

new_df.to_excel('./Datap3p_Label2.xlsx', index=False)
print('Total data :', len(new_df))
new_df.head(10)

# ============================================================== #

# TF-IDF

In [ ]:
import pandas as pd 
import numpy as np

# TWEET_DATA = pd.read_csv("Text_Preprocessing.csv", usecols=["label", "tweet_tokens_stemmed"])
# TWEET_DATA.columns = ["label", "tweet"]

TWEET_DATA = pd.read_excel('./tes_label.xlsx', usecols=["Tweets", "Label"])
TWEET_DATA.columns = ["tweet", "label"]

TWEET_DATA

In [ ]:
import ast

def convert_text_list(texts):
    texts = ast.literal_eval(texts)
    return [text for text in texts]

# TWEET_DATA["tweet_list"] = TWEET_DATA["tweet"].apply(convert_text_list)

def Tokenization(tweetsText):
    tweetsText = tweetsText.split()
    return tweetsText

TWEET_DATA["tweet_list"] = TWEET_DATA["tweet"].apply(Tokenization)

print(TWEET_DATA["tweet_list"])

print("\ntype : ", type(TWEET_DATA["tweet_list"]))

In [ ]:
def calc_TF(document):
    # Counts the number of times the word appears in review
    TF_dict = {}
    for term in document:
        if term in TF_dict:
            TF_dict[term] += 1
        else:
            TF_dict[term] = 1
    # Computes tf for each word
    for term in TF_dict:
        TF_dict[term] = TF_dict[term] / len(document)
    return TF_dict

TWEET_DATA["TF_dict"] = TWEET_DATA['tweet_list'].apply(calc_TF)

TWEET_DATA["TF_dict"][10]

In [ ]:
# Check TF result
index = 0

print('%20s' % "term", "\t", "TF\n")
for key in TWEET_DATA["TF_dict"][index]:
    print('%20s' % key, "\t", TWEET_DATA["TF_dict"][index][key])

In [ ]:
def calc_DF(tfDict):
    count_DF = {}
    # Run through each document's tf dictionary and increment countDict's (term, doc) pair
    for document in tfDict:
        for term in document:
            if term in count_DF:
                count_DF[term] += 1
            else:
                count_DF[term] = 1
    return count_DF

DF = calc_DF(TWEET_DATA["TF_dict"])
DF

In [ ]:
n_document = len(TWEET_DATA)

def calc_IDF(__n_document, __DF):
    IDF_Dict = {}
    for term in __DF:
        IDF_Dict[term] = np.log(__n_document / (__DF[term] + 1))
    return IDF_Dict
  
#Stores the idf dictionary
IDF = calc_IDF(n_document, DF)
IDF

In [ ]:
#calc TF-IDF
def calc_TF_IDF(TF):
    TF_IDF_Dict = {}
    #For each word in the review, we multiply its tf and its idf.
    for key in TF:
        TF_IDF_Dict[key] = TF[key] * IDF[key]
    return TF_IDF_Dict

#Stores the TF-IDF Series
TWEET_DATA["TF-IDF_dict"] = TWEET_DATA["TF_dict"].apply(calc_TF_IDF)
TWEET_DATA["TF-IDF_dict"]

In [ ]:
# Check TF-IDF result
index = 0

print('%20s' % "term", "\t", '%10s' % "TF", "\t", '%20s' % "TF-IDF\n")
for key in TWEET_DATA["TF-IDF_dict"][index]:
    print('%20s' % key, "\t", TWEET_DATA["TF_dict"][index][key] ,"\t" , TWEET_DATA["TF-IDF_dict"][index][key])

In [ ]:
# sort descending by value for DF dictionary 
sorted_DF = sorted(DF.items(), key=lambda kv: kv[1], reverse=True)[:50]

# Create a list of unique words from sorted dictionay `sorted_DF`
unique_term = [item[0] for item in sorted_DF]

def calc_TF_IDF_Vec(__TF_IDF_Dict):
    TF_IDF_vector = [0.0] * len(unique_term)

    # For each unique word, if it is in the review, store its TF-IDF value.
    for i, term in enumerate(unique_term):
        if term in __TF_IDF_Dict:
            TF_IDF_vector[i] = __TF_IDF_Dict[term]
    return TF_IDF_vector

TWEET_DATA["TF_IDF_Vec"] = TWEET_DATA["TF-IDF_dict"].apply(calc_TF_IDF_Vec)

print("print first row matrix TF_IDF_Vec Series\n")
print(TWEET_DATA["TF_IDF_Vec"])

print("\nmatrix size : ", len(TWEET_DATA["TF_IDF_Vec"]))

In [ ]:
# Convert Series to List
TF_IDF_Vec_List = np.array(TWEET_DATA["TF_IDF_Vec"].to_list())

# Sum element vector in axis=0 
sums = TF_IDF_Vec_List.sum(axis=0)

data = []

for col, term in enumerate(unique_term):
    data.append((term, sums[col]))
    
ranking = pd.DataFrame(data, columns=['term', 'rank'])
ranking.sort_values('rank', ascending=False)

# ============================================================== #

In [ ]:
df.isnull().sum()

In [ ]:
df['Label'].value_counts()
df = df[df['Label'] != 'netral']

In [ ]:
import numpy as np

df["Positif"] = np.where(df["Label"] > 3, 1, 0)

In [ ]:
import seaborn as sns

sns.countplot(df["Positif"])

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidVectorizer

X_train, X_test, y_train, y_test = train_test_split(df["Tweets"]. df["Positif"], random_state=50)

vect = TfidVectorizer().fit(X_train)
len(vect.get_feature_names())

In [ ]:
from sklearn.linear_model import LogisticRegression

X_train_vectorized = vect.transform(X_train)

model = LogisticRegression()
model.fit(X_train_vectorized, y_train)

predictions = model.predict(vect.transform(X_test))

In [ ]:
from sklearn.metrics import roc_auc_score

print("AUC : ", roc_auc_score(y_test, predictions))

feature_names = np.array(vect.get_feature_names())
sorted_coef_index = model.coef_[0].argsort()

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer  
tfidfconverter = TfidfVectorizer(max_features=2000, min_df=10, max_df=0.9)
X = tfidfconverter.fit_transform(df_label).toarray()
 
from sklearn.model_selection import train_test_split  
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
 
 
from sklearn.ensemble import RandomForestClassifier
text_classifier = RandomForestClassifier(n_estimators=100, random_state=0)  
text_classifier.fit(X_train, y_train)


predictions = text_classifier.predict(X_test)
 
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
 
print(confusion_matrix(y_test,predictions))  
print(classification_report(y_test,predictions))  
print(accuracy_score(y_test, predictions))

In [ ]:
import pandas as pd

# df_label = pd.read_excel('./UjiRandomCodes/test_label.xlsx', engine='openpyxl')
# df_label = pd.read_csv('./UjiRandomCodes/test_label.csv', sep=";")
df_label = pd.read_excel('./Datap3p_Preprocessed.xlsx', engine='openpyxl', sheet_name="Stemming")
df_label

In [ ]:
# resultAnalysis
tweets_positive = [t for t in df_label["Label"] if t == "Positif"]
tweets_negative = [t for t in df_label["Label"] if t == "Negatif"]
tweets_netral = [t for t in df_label["Label"] if t == "Netral"]

print("Hasil Sentimen Analisis")
print("Positif : ", len(tweets_positive), "({}%)".format(100*len(tweets_positive)/len(df_label)))
print("Negatif : ", len(tweets_negative), "({}%)".format(100*len(tweets_negative)/len(df_label)))
print("Netral  : ", len(tweets_netral), "({}%)".format(100*len(tweets_netral)/len(df_label)))

In [ ]:
df_label2 = df_label.astype({'Label' : 'category'})
df_label2 = df_label.astype({'Tweets' : 'string'})
df_label2.dtypes

In [ ]:
# Proses TF-IDF

from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

tvec = CountVectorizer()
X_cVec = tvec.fit_transform(df_label['Tweets'].values.astype('U'))
# print(X_cVec)
h_tfidf = TfidfTransformer()
x_tfidf = h_tfidf.fit_transform(X_cVec)
# print(x_tfidf)
X = df_label.Tweets
Y = df_label.Label

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

df_label = pd.read_excel('./Datap3p_Label.xlsx', engine='openpyxl')

# text_tf = TfidfVectorizer().fit_transform(df_label['Tweets'].astype('U'))

# X_train, X_test, y_train, y_test = train_test_split(text_tf, df_label['Label'], test_size = 0.2, random_state = 43)


def pre_processing(df):

    """ partioning data into features and target """

    X = df.drop([df.columns[-1]], axis = 1)
    y = df[df.columns[-1]]

    return X, y

def train_test_split(x, y, test_size = 0.2, random_state = None):

    """ partioning the data into train and test sets """

    x_test = x.sample(frac = test_size, random_state = random_state)
    y_test = y[x_test.index]

    x_train = x.drop(x_test.index)
    y_train = y.drop(y_test.index)

    return x_train, x_test, y_train, y_test

#Split fearures and target
X,y  = pre_processing(df_label)

#Split data into Training and Testing Sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 30)


X_train

In [ ]:
class multinomialNB:
    def __init__(self, alpha=1):
        self.alpha = alpha 

    def fit(self, X_train, y_train):
        m, n = X_train.shape
        self._classes = np.unique(y_train)
        n_classes = len(self._classes)

        # init: Prior & Likelihood
        self._priors = np.zeros(n_classes)
        self._likelihoods = np.zeros((n_classes, n))

        # Get Prior and Likelihood
        for idx, c in enumerate(self._classes):
            X_train_c = X_train[c == y_train]
            self._priors[idx] = X_train_c.shape[0] / m 
            self._likelihoods[idx, :] = ((X_train_c.sum(axis=0)) + self.alpha) / (np.sum(X_train_c.sum(axis=0) + self.alpha))


    def predict(self, X_test):
        return [self._predict(x_test) for x_test in X_test]

    def _predict(self, x_test):
        # Calculate posterior for each class
        posteriors = []
        for idx, c in enumerate(self._classes):
            prior_c = np.log(self._priors[idx])
            likelihoods_c = self.calc_likelihood(self._likelihoods[idx,:], x_test)
            posteriors_c = np.sum(likelihoods_c) + prior_c
            posteriors.append(posteriors_c)

        return self._classes[np.argmax(posteriors)]

    def calc_likelihood(self, cls_likeli, x_test):
        return np.log(cls_likeli) * x_test

    def score(self, X_test, y_test):
        y_pred = self.predict(X_test)
        return np.sum(y_pred == y_test)/len(y_test)

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

df_label = pd.read_excel('./tes_label.xlsx', engine='openpyxl')

tf = TfidfVectorizer()
text_tf = tf.fit_transform(df_label['Tweets'].astype('U'))

# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 30)

X_train, X_test, y_train, y_test = train_test_split(text_tf, df_label['Label'], test_size = 0.2, random_state = 43)

MNB = multinomialNB()
MNB_fit = MNB.fit(X_train, y_train)
# print(text_tf)

In [ ]:
class MultinomialNB():

    def fit(self, X_train, y_train, ls=0.01):
        self.ls = ls
        self.y_classes, y_counts = np.unique(y_train, return_counts=True)
        self.x_classes = [np.unique(x) for x in X_train]
        self.phi_y = 1.0 * y_counts/y_counts.sum()
        self.phi_x = self.mean_X(X_train, y_train)
        self.c_x = self.count_x(X_train, y_train)
        return self

    def mean_X(self, X_train, y_train):
        return [[self.ls_mean_x(X_train, y_train, k, j) for j in range(len(self.x_classes))] for k in self.y_classes]

    def ls_mean_x(self, X_train, y_train, k, j):
        x_data = (X_train[:,j][y_train==k].reshape(-1,1) == self.x_classes[j])
        return (x_data.sum(axis=0) + self.ls ) / (len(x_data) + (len(self.x_classes) * self.ls))

    def get_mean_x(self, y_train, j):
        return 1 + self.ls / (self.c_x[y][j] + (len(self.x_classes) * self.ls))

    def count_x(self, X_train, y_train):
        return [[len(X[:,j][y_train==k].reshape(-1,1) == self.x_classes[j])
                       for j in range(len(self.x_classes))]
                      for k in self.y_classes]

    def predict(self, X_train):
        return np.apply_along_axis(lambda x: self.compute_probs(x), 1, X_train)

    def compute_probs(self, x):
        probs = np.array([self.compute_prob(x, y) for y in range(len(self.y_classes))])
        return self.y_classes[np.argmax(probs)]

    def compute_prob(self, x, y):
        Pxy = 1
        for j in range(len(x)):
            x_clas = self.x_classes[j]
            if x[j] in x_clas:
                i = list(x_clas).index(x[j])
                p_x_j_y = self.phi_x[y][j][i] # p(xj|y)
                Pxy *= p_x_j_y
            else:
                Pxy *= get_mean_x(y, j)
        return Pxy * self.phi_y[y]

    def evaluate(self, X_train, y_train):
        return (self.predict(X_train) == y_train).mean()

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

df_label = pd.read_excel('./tes_label.xlsx', engine='openpyxl')

tf = TfidfVectorizer()
text_tf = tf.fit_transform(df_label['Tweets'].astype('U'))

# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 30)

X_train, X_test, y_train, y_test = train_test_split(text_tf, df_label['Label'], test_size = 0.2, random_state = 43)

MNB = MultinomialNB()
MNB_fit = MNB.fit(X_train, y_train)
MNB_pred = MNB.predict(X_test)
print(MNB_fit)


In [ ]:
import pandas as pd
import numpy as np

df_label = pd.read_excel('./Datap3p_Preprocessed.xlsx', engine='openpyxl', sheet_name="Stemming")

# TF-IDF
from sklearn.feature_extraction.text import TfidfVectorizer

tf = TfidfVectorizer()
text_tf = tf.fit_transform(df_label['Tweets'])

# Splitting Data
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(text_tf, df_label['Label'], test_size = 0.1, random_state = 23)
type(X_train)

In [ ]:
# learning the weights of the features
def calc_cond_probs(X_train, dirichlet_alpha):
    features_y0 = []
    features_y1 = []
    
    # calculating the total number of words found in each class across all features
    vocab_y0 = np.sum(X_train[:350,:])
    vocab_y1 = np.sum(X_train[350:,:])

    # calculating conditional probabilities for each feature for each class assuming a Dirichlet prior for smoothing
    for i in range(X_train.shape[1]):
            temp_y0_feat = (np.sum(X_train[:350, i]) + dirichlet_alpha)/(vocab_y0 + dirichlet_alpha*X_train.shape[1])
            features_y0.append(temp_y0_feat)

            temp_y1_feat = (np.sum(X_train[350:, i]) + dirichlet_alpha)/(vocab_y1 + dirichlet_alpha*X_train.shape[1])
            features_y1.append(temp_y1_feat)

    return np.array(features_y0), np.array(features_y1)

# the the hyperparameter dirichlet_alpha was set to be 1
# dirichlet_alpha is not yet optimized
w_y0, w_y1 = calc_cond_probs(X_train, dirichlet_alpha=1)

print("Feature weights for class 0:", w_y0[:5])
print("Feature weights for class 1:", w_y1[:5])

In [ ]:
def calc_log_likelihood(f_y0, f_y1, x):

    log_likelihood_y0 = []
    log_likelihood_y1 = []

    # calculating log likelihoods using weights of features for the dataset of interest 
    for i in range(x.shape[0]):
        log_likelihood_y0.append(sum(np.log(f_y0)*x[i, :]))
        log_likelihood_y1.append(sum(np.log(f_y1)*x[i, :]))

    return log_likelihood_y0, log_likelihood_y1

# calculating log likelikhoods for the training dataset
log_w_y0, log_w_y1 = calc_log_likelihood(w_y0, w_y1, X_train)

print("Log likelihoods of observations for class 0:", [round(number, 2) for number in log_w_y0[:5]])
print("Log likelihoods of observations for class 1:", [round(number, 2) for number in log_w_y1[:5]])

In [ ]:
# Performa Algoritma Naive Bayes
from sklearn.naive_bayes import MultinomialNB, GaussianNB, BernoulliNB
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

clf = MultinomialNB().fit(X_train, y_train)
predicted = clf.predict(X_test)

# clf = GaussianNB().fit(X_train.toarray(), y_train)
# predicted = clf.predict(X_test.toarray())
print("MultinomialNB Accuracy : ", accuracy_score(y_test, predicted))
print("MultinomialNB Precision : ", precision_score(y_test, predicted, average = "micro", pos_label = "Negatif"))
print("MultinomialNB Recall : ", recall_score(y_test, predicted, average = "micro", pos_label = "Negatif"))
print("MultinomialNB f1_score : ", f1_score(y_test, predicted, average = "micro", pos_label = "Negatif"))

print(f'confusion matrix:\n {confusion_matrix(y_test, predicted)}')
print("========================================================\n")
print(classification_report(y_test, predicted, zero_division = 0))

# ============================================================== #

In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Mon Dec 31 14:49:57 2018
 
@author: Usman
"""
 
 
 
import numpy as np 
import pandas as pd 
import re  
import nltk  
nltk.download('stopwords')  
from nltk.corpus import stopwords  
 
# tweets = pd.read_csv("https://raw.githubusercontent.com/kolaveridi/kaggle-Twitter-US-Airline-Sentiment-/master/Tweets.csv")
tweets = pd.read_excel('./Datap3p_14Label.xlsx')
 
tweets.head()
 
tweets.shape
 
 
import seaborn as sns
sns.countplot(x='Label', data=tweets)
 
sns.countplot(x='Tweets', data=tweets)
 
sns.countplot(x='Tweets', hue="Label", data=tweets)
 
 
X = tweets.iloc[:, 10].values  
y = tweets.iloc[:, 1].values
 
 
processed_tweets = []
 
for tweet in range(0, len(X)):  
    # Remove all the special characters
    processed_tweet = re.sub(r'\W', ' ', str(X[tweet]))
 
    # remove all single characters
    processed_tweet = re.sub(r'\s+[a-zA-Z]\s+', ' ', processed_tweet)
 
    # Remove single characters from the start
    processed_tweet = re.sub(r'\^[a-zA-Z]\s+', ' ', processed_tweet) 
 
    # Substituting multiple spaces with single space
    processed_tweet= re.sub(r'\s+', ' ', processed_tweet, flags=re.I)
 
    # Removing prefixed 'b'
    processed_tweet = re.sub(r'^b\s+', '', processed_tweet)
 
    # Converting to Lowercase
    processed_tweet = processed_tweet.lower()
 
    processed_tweets.append(processed_tweet)
    
    
 
from sklearn.feature_extraction.text import TfidfVectorizer  
tfidfconverter = TfidfVectorizer(max_features=2000, min_df=5, max_df=0.7, stop_words=stopwords.words('english'))  
X = tfidfconverter.fit_transform(processed_tweets).toarray()
 
from sklearn.model_selection import train_test_split  
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
 
 
from sklearn.ensemble import RandomForestClassifier
text_classifier = RandomForestClassifier(n_estimators=100, random_state=0)  
text_classifier.fit(X_train, y_train)
 
 
predictions = text_classifier.predict(X_test)
 
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
 
print(confusion_matrix(y_test,predictions))  
print(classification_report(y_test,predictions))  
print(accuracy_score(y_test, predictions))
 

In [ ]:
X

In [ ]:
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

def stemming_text_1():
    with open('test.txt', 'r') as f:
        for line in f:
            print (line)
            singles = []

            factory = StemmerFactory()
            stemmer = factory.create_stemmer()
            for plural in line.split():
                singles.append(stemmer.stem(plural))
            print (' '.join(singles))
            
stemming_text_1()

In [ ]:
# Basic Preprocessing Text
import numpy as np
import re

stopwords = ["for", "on", "an", "a", "of", "and", "in", "the", "to", "from"]


def clean_tweet(tweet):
    temp = tweet.lower()
#     tweetsText = ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)"," ", tweetsText).split())
    temp = re.sub("'", "", temp) # to avoid removing contractions in english
    temp = re.sub("@[A-Za-z0-9_]+","", temp)
    temp = re.sub("#[A-Za-z0-9_]+","", temp)
    temp = re.sub(r'http\S+', '', temp)
    temp = re.sub('[()!?]', ' ', temp)
    temp = re.sub('\[.*?\]',' ', temp)
    temp = re.sub("[^a-z0-9]"," ", temp)
    temp = temp.split()
    temp = [w for w in temp if not w in stopwords]
    temp = " ".join(word for word in temp)
    return temp

def Cleansing(tweetsText):
    tweetsText = tweetsText.lower()
    
    # Menghilangkan mention, hashtag, character reference
    tweetsText = re.sub('[@#&][A-Za-z0-9_]+'," ", tweetsText)

    # Menghilangkan tautan
    tweetsText = re.sub("\w+:\/\/\S+"," ", tweetsText)
    
    # Menghilangkan tanda baca
    tweetsText = re.sub('[()!?;,]', ' ', tweetsText)
    tweetsText = re.sub('\[.*?\]',' ', tweetsText)

    # Menghilangkan tanda selain huruf dan angka
    tweetsText = re.sub("[^a-z0-9]"," ", tweetsText)
    
    pattern = re.compile(r"(.)\1{1,}", re.DOTALL)
    tweetsText=  pattern.sub(r"\1\1", tweetsText)

    
    # Menghilangkan spasi lebih dari 1
    tweetsText = ' '.join(tweetsText.split())
    return tweetsText

tweets = ["&amp; Gett Gett rrrrready for #NatGeoEarthDay! Join2 us on 4/21 for an evening of music and celebration, exploration and inspiration https://on.natgeo.com/3t0wzQy.",
"Coral in the shallows of Aitutaki Lagoon, Cook Islands, Polynesia https://on.natgeo.com/3gkgq4Z",
"Don't miss our @reddit_id AMA with author and climber Mark Synnott who will be answering your questions about his historic journey to the North Face of Everest TODAY at 12:00pm ET! Start submitting your questions here: https://on.natgeo.com/3ddSkHk @DuttonBooks"]
 
results = [Cleansing(tw) for tw in tweets]
results

In [ ]:
import re
tweetsText = "    Get ready for #NatGeoEarthDay_&!        , Join us on? 4/2 @_Reks_bl https://on.natgeo.com/3gkgq4Z 12:00 WIB."
tweetsText = tweetsText.lower()

# Menghilangkan tanda @ (mention username)
# tweetsText = re.sub('@[A-Za-z0-9_]+'," ", tweetsText)

#Menghilangkan tanda # (hashtag tiap tweet)
# tweetsText = re.sub('#[A-Za-z0-9_]+'," ", tweetsText)

# tweetsText = re.sub(r'http\S+', ' ', tweetsText)

# tweetsText = re.sub('[()!?;,]', ' ', tweetsText)
# tweetsText = re.sub('\[.*?\]',' ', tweetsText)

tweetsText = re.sub("[^a-z0-9]"," ", tweetsText)

# Menghilangkan Tautan
# tweetsText = re.sub("\w+:\/\/\S+"," ", tweetsText)

tweetsText = re.sub("\d+"," ", tweetsText)

# Menghilangkan 2 atau lebih spasi
# tweetsText = re.sub("\s+"," ", tweetsText)
tweetsText = " ".join(word for word in tweetsText.split())

print(tweetsText)

In [ ]:

# Basic Replace Slangwords
the_string="That is mine that is yours"

the_dictionary={'mine': 'punya gw', 'I': 'saya', 'yours': 'punya lu', 'you': 'I'}

list_string = the_string.split()

for i,word in enumerate(list_string):
    if word in the_dictionary:
        list_string[i] = the_dictionary[word]

print(' '.join(list_string))

In [ ]:
#=================================================================================================================#

In [ ]:
dataTweets = api.search_tweets(q='hepatitis -filter:retweets', tweet_mode='extended', lang='id', count=3)

for tweet in dataTweets:
    print(tweet.id, str(tweet.created_at.date()) + ' '+ str(tweet.created_at.time()), tweet.full_text, tweet.user.screen_name)
    print()
    

In [ ]:
dataTweets = api.search_tweets(q=', '.join(keyword) + ' -filter:retweets', max_id = 1510854677855756288, tweet_mode='extended', lang='id', count=3)

for tweet in dataTweets:
    print(tweet.id, str(tweet.created_at.date()) + ' '+ str(tweet.created_at.time()), tweet.full_text, tweet.user.screen_name)
    print()
    

In [ ]:
dataTweets = api.search_tweets(q=', '.join(keyword) + ' -filter:retweets', max_id = 1510854677855756288, tweet_mode='extended', lang='id', count=3)

for tweet in dataTweets:
    print(tweet.id, tweet.created_at, tweet.full_text, tweet.user.screen_name)
    print()

In [ ]:
#import datetime
from datetime import datetime
from datetime import timedelta

for tweet in dataTweets:
    

    clean_timestamp = datetime.strptime(str(tweet.created_at.date()) + ' '+ str(tweet.created_at.time()),
                       '%Y-%m-%d %H:%M:%S')
    offset_hours = +7 #offset in hours for EST timezone

    #account for offset from UTC using timedelta                                
    local_timestamp = clean_timestamp + timedelta(hours=offset_hours)
    
    #convert to am/pm format for easy reading
    final_timestamp =  datetime.strftime(local_timestamp, '%Y-%m-%d %H:%M:%S')
    
    print(tweet.id, final_timestamp, tweet.full_text, tweet.user.screen_name)
    print()

In [ ]:
from datetime import datetime
import time

def datetime_from_utc_to_local(utc_datetime):
    now_timestamp = time.time()
    offset = datetime.fromtimestamp(now_timestamp) - datetime.utcfromtimestamp(now_timestamp)
    return utc_datetime + offset

In [ ]:
from datetime import datetime
from dateutil import tz

# METHOD 2: Auto-detect zones:
from_zone = tz.tzutc()
to_zone = tz.tzlocal()

# utc = datetime.utcnow()
utc = datetime.strptime('2022-04-04 08:06:57', '%Y-%m-%d %H:%M:%S')

# Tell the datetime object that it's in UTC time zone since 
# datetime objects are 'naive' by default
utc = utc.replace(tzinfo=from_zone)

# Convert time zone
central = utc.astimezone(to_zone)

In [ ]:
central

In [ ]:
utc

In [ ]:
import datetime
utc_datetime = datetime.datetime.utcnow()
utc_datetime.strftime("%Y-%m-%d %H:%M:%S")

In [ ]:
from datetime import datetime
from email.utils import parsedate_tz, mktime_tz

def to_local_time(tweet_time_string):
    """Convert rfc 5322 -like time string into a local time
       string in rfc 3339 -like format.

    """
    timestamp = mktime_tz(parsedate_tz(tweet_time_string))
    return datetime.fromtimestamp(timestamp).strftime('%Y-%m-%d %H:%M:%S')

time_string = to_local_time('2022-04-04 05:38:36+0000')
# use time_string here..

In [ ]:
import numpy as np
rng = np.random.RandomState(1)
X = rng.randint(5, size=(6, 100))
y = np.array([1, 2, 3, 4, 5, 6])
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB()
clf.fit(X, y)
X